<a href="https://colab.research.google.com/github/prabal5ghosh/Deep-Learning-summer-school-2025-university-of-cote-d-Azur/blob/main/TPtokenizer_SUJET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prabal Ghosh  

# date- 23/06/2025

<center><img width=60% src="http://www.i3s.unice.fr/~lingrand/efeliaUnica.png"><br/><br/>
<font size=+3><b>TP tokenization</b></font><br/><br/>
    <font size=+1>Diane Lingrand, Célia D'cruz and Frédéric Precioso<br/><br/>
    2025 - June/July</font><br/>
    <img width=14% src="http://www.i3s.unice.fr/~lingrand/cc-long.png">
    </center>


In this notebook, you will experiment one very simple tokenizer and the tokenizer that is part of GPT family: BPE.

The idea is to rewrite these algorithms in order to deeply understand their principles (and limitations) before using more effective implementation using librairies such as `tiktoken` or from Hugging Face.

In [1]:
# necessary imports
import re # for regular expressions
import numpy as np
from collections import defaultdict

# A vanilla tokenizer

In this section, we will build a very simple tokenizer that aims at experimenting basics ideas and problems. Usually, tokenizers are composed of two parts:
- the pre-tokenizer that splits a text into tokens (letters, words ...)
- a tokenizer that performs different operations such as merging, splitting, counting occurrences... and assigns a numerical code to each final token.


Our vanilla tokenizer will split a text into words, build a dictionary and encode each token by its index in the dictionary.


Example: "Welcome to the 2025 Deep Learning School! Deep Learning is fun. Hope you will learn a lot at the school!" lead to the dictionary containing:

<table><tr><td>welcome</td><td>to</td><td>the</td><td>2025</td><td>deep</td><td>learning</td><td>school</td><td>is</td><td>fun</td><td>hope</td><td>you</td><td>will</td><td>learn</td><td>a</td><td>lot</td><td>at</td><td>!</td></tr>
    <tr><td>0 </td><td>1 </td><td>2 </td><td>3 </td><td>4 </td><td>5 </td><td>6</td><td> 7 </td><td>8 </td><td>9 </td><td>10 </td><td>11</td><td> 12</td><td> 13</td><td> 14</td><td> 15 </td><td>16</td><td>17</td></tr></table>

Thus, the encoding of the previous sentence becomes:

<pre>0 1 2 3 4 5 6 16 4 5 7 8 17 9 10 11 12 13 14 15 2 6 16</pre>

## Splitting a text into words and punctuations

For this part, you will use regular expression using the [`re`](https://docs.python.org/3/library/re.html) library. As you may have noticed, we remove the space character in order to save space. Be aware that in some applications, it could be necessary to add the space character to the dictionary and to encode it.

🤔 <b><font color='purple'>Question:</font></b> Using the [`lower` function](https://docs.python.org/3.3/library/stdtypes.html#str.lower), convert the string below into lower case letters. Then, using the [`split` function](https://docs.python.org/3/library/re.html#re.split), split the resulting text into words AND punctuations. For example, "Deep Learning School!!!" should be converted to ["deep", "learning", "school", "!", "!", "!"]. Start by lowercasing the text.

In [2]:
text = "Welcome to the 2025 Deep Learning School! Deep Learning is fun. Hope you will learn a lot at the school!" # this text will be reused multiple times in the beginning of the lab

# your work

In [3]:
# hint 1: lower letters
textLower = text.lower()
print(textLower)

welcome to the 2025 deep learning school! deep learning is fun. hope you will learn a lot at the school!


In [4]:
# hint 2: re.split usage
# in this example, the splitting is done for every space (and only spaces)
textSplitted = re.split(r'(\s)',textLower)
print(textSplitted)
print()
print()
# splitting with everything not a letter or a digit (space, punctuation ...)
textSplitted2 = re.split(r'[^a-z0-9]',textLower)
print(textSplitted2)
print()
print()
# splitting using punctuations only
textSplitted3 = re.split(r'[,.!;:?]', textLower)
print(textSplitted3)

['welcome', ' ', 'to', ' ', 'the', ' ', '2025', ' ', 'deep', ' ', 'learning', ' ', 'school!', ' ', 'deep', ' ', 'learning', ' ', 'is', ' ', 'fun.', ' ', 'hope', ' ', 'you', ' ', 'will', ' ', 'learn', ' ', 'a', ' ', 'lot', ' ', 'at', ' ', 'the', ' ', 'school!']


['welcome', 'to', 'the', '2025', 'deep', 'learning', 'school', '', 'deep', 'learning', 'is', 'fun', '', 'hope', 'you', 'will', 'learn', 'a', 'lot', 'at', 'the', 'school', '']


['welcome to the 2025 deep learning school', ' deep learning is fun', ' hope you will learn a lot at the school', '']


🤔 <b><font color='purple'>Question:</font></b> Compute the list of unique tokens (lowercased words and punctuations) of the text from the beginning of the lab using the [`np.unique`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) function.

In [5]:
import numpy as np

In [6]:
# your work
unique_tokens = np.unique(textSplitted2)
print(unique_tokens)

['' '2025' 'a' 'at' 'deep' 'fun' 'hope' 'is' 'learn' 'learning' 'lot'
 'school' 'the' 'to' 'welcome' 'will' 'you']


In [7]:
# your work
unique_tokens = np.unique(textSplitted2)
print(unique_tokens)

['' '2025' 'a' 'at' 'deep' 'fun' 'hope' 'is' 'learn' 'learning' 'lot'
 'school' 'the' 'to' 'welcome' 'will' 'you']


🤔 <b><font color='purple'>Question:</font></b> Using the list of unique tokens, create the dictionary `token2id` that associates each unique token (dictionary key) to an unique ID (dictionary value): the beginning of the lab provides an example of the dictionary. Then, create the `id2token` dictionary that peforms the inverse associations.

In [8]:
# your work
token2id = {token: i for i, token in enumerate(unique_tokens)}
id2token = {i: token for i, token in enumerate(unique_tokens)}

print("token2id:", token2id)
print("\nid2token:", id2token)

token2id: {np.str_(''): 0, np.str_('2025'): 1, np.str_('a'): 2, np.str_('at'): 3, np.str_('deep'): 4, np.str_('fun'): 5, np.str_('hope'): 6, np.str_('is'): 7, np.str_('learn'): 8, np.str_('learning'): 9, np.str_('lot'): 10, np.str_('school'): 11, np.str_('the'): 12, np.str_('to'): 13, np.str_('welcome'): 14, np.str_('will'): 15, np.str_('you'): 16}

id2token: {0: np.str_(''), 1: np.str_('2025'), 2: np.str_('a'), 3: np.str_('at'), 4: np.str_('deep'), 5: np.str_('fun'), 6: np.str_('hope'), 7: np.str_('is'), 8: np.str_('learn'), 9: np.str_('learning'), 10: np.str_('lot'), 11: np.str_('school'), 12: np.str_('the'), 13: np.str_('to'), 14: np.str_('welcome'), 15: np.str_('will'), 16: np.str_('you')}


🤔 <b><font color='purple'>Question:</font></b> Using the previous dictionaries, encode the text from the beginning of the lab. Display the encoded text. It should be a list of IDs, where the Nth ID in this list represents the Nth word or punctuation in the initial text. The beginning of the lab provides an example of the encoded text.

In [9]:
text

'Welcome to the 2025 Deep Learning School! Deep Learning is fun. Hope you will learn a lot at the school!'

In [10]:
# your work
encoded_text = [token2id[token] for token in textSplitted2]
print(encoded_text)

[14, 13, 12, 1, 4, 9, 11, 0, 4, 9, 7, 5, 0, 6, 16, 15, 8, 2, 10, 3, 12, 11, 0]


🤔 <b><font color='purple'>Question:</font></b> Using the previous dictionaries, perform a simple decoding of the encoded text (list of IDs) from the previous question, as an first rough attempt to reconstruct the initial text.

In [11]:
# your work
decoded_text = ' '.join([id2token[id] for id in encoded_text])
print(decoded_text)

welcome to the 2025 deep learning school  deep learning is fun  hope you will learn a lot at the school 


In [12]:
# your work
decoded_text = ''
for id in encoded_text:
    decoded_text += ' ' + id2token[id]
print(decoded_text)

 welcome to the 2025 deep learning school  deep learning is fun  hope you will learn a lot at the school 


In [13]:
text

'Welcome to the 2025 Deep Learning School! Deep Learning is fun. Hope you will learn a lot at the school!'

🤔 <b><font color='purple'>Question:</font></b> Are you satisfied with the decoding? What information was lost during the process of pretokenizing and tokenizing the initial text?

No the decoding is not able to generate the initial text properly. the decoded text does not contain any ponctuation.  example ! .

as duing tokenization we remove the punctuation.

In [29]:
# your answer
# ...

# your code to perform another attempt that is more complex to better decode the text: use common English rules to make up for the loss of information
# ...

# textLower = text.lower()
textLower = text

# print(textLower)
res = re.findall( r'\w+|[^\s\w]+', textLower)
# print(res)

# your work
unique_tokens = np.unique(res)
# print(unique_tokens)

# your work
token2id = {token: i for i, token in enumerate(unique_tokens)}
id2token = {i: token for i, token in enumerate(unique_tokens)}

# print("token2id:", token2id)
# print("\nid2token:", id2token)

# your work
encoded_text = [token2id[token] for token in res]
# print(encoded_text)

decoded_text = ''
for id in encoded_text:
  if id2token[id] == "." or id2token[id] == "!":
      decoded_text += id2token[id]+ ' '
  else:
      decoded_text += ' ' + id2token[id]

print(decoded_text)

print()
print()
print(text)

 Welcome to the 2025 Deep Learning School!  Deep Learning is fun.  Hope you will learn a lot at the school! 


Welcome to the 2025 Deep Learning School! Deep Learning is fun. Hope you will learn a lot at the school!


## Writing a python class for encoding / decoding

🤔 <b><font color='purple'>Question:</font></b> Organize all the previous pieces of code into a python object representing this tokenizer. Write a constructor that will build the dictionnaries from a corpus (list of strings). Then, write an encode and a decode methods.

In [ ]:
# your work

In [ ]:
# hint: the skeleton
class VanillaTokenizer:

    def __init__(self, corpus):
        # corpus is a list of texts. Concatenate the texts. It will be used to create the dictionaries.
        # Do all the necessary steps to create the dictionaries
        self.token2id =
        self.id2token =

    def encode(self, text):
        # We want to create an ordered list of IDs that corresponds to the encoded text (Nth ID in this list represents the Nth word or punctuation in the text)
        # Do all the necessary steps to create this list, and return it
        return ids

    def decode(self, ids):
        # the "ids" parameter corresponds to the encoded text (ordered list of IDs). We want to decoded it to get the initial text.
        # Do all the necessary steps to create this decoded text, and return it
        return text


🤔 <b><font color='purple'>Question:</font></b> Using the class VanillaTokenizer, build a tokenizer object initialized from the text at the beginning of the lab: "Welcome to the 2025 Deep Learning School! Deep Learning is fun. Hope you will learn a lot at the school!". Then, try to encode the 2 following texts: "Deep Learning School is fun!" and "Welcome to Sophia Antipolis!"

In [ ]:
txt1 = "Deep Learning School is fun."
txt2 = "Welcome to Sophia Antipolis!"

# your answer

You may encounter an error of type `KeyError` about the token `sophia`  because 'Sophia Antipolis' is not in the vocabulary. The next question is about dealing with unknowns tokens.

🤔 <b><font color='purple'>Question:</font></b> Modify the class by adding a entry to the dictionary corresponding to unknown tokens: `[UNK]`

In [ ]:
# your work

In [ ]:
# hint 1: outside the class
token2id['[UNK]'] = len(token2id)
id2token[len(id2token)] = '[UNK]'

In [ ]:
# hint 2: encoding modified
# Add a step that replace every unknown token in textSplitted by [UNK]

🤔 <b><font color='purple'>Question:</font></b> Decode the two encoded texts

In [ ]:
# your work

# BPE Algorithm

## Vocabulary for the BPE algorithm

🤔 <b><font color='purple'>Question:</font></b> For the BPE used in GPT2, spaces have been replaced by the character 'Ġ' while these spaces have remain unchanged in more recent versions of GPT. For clarity in this exercise, will keep the  'Ġ' in replacement of spaces. The vocabulary is initialized by all letters and symbols appearing in the corpus of texts. This time, we do not split by word, but by character: the initial vocabulary might look like ["[EOT]", "!", ",", "0", "a", "b", "Ġ"] (this is an example, not the real answer). Let's start with the previous corpus and compute the vocabulary!


In [ ]:
corpus = [text]

In [ ]:
alphabet = []

# your work is to add (only once) all letters and symbols appearing in the corpus
# don't forget to add the symbol of end of text.
vocab = ["[UNK]","[EOT]"] + alphabet.copy()


## Enriching the corpus
We enrich the corpus with a [small english text: The Story of an Hour](https://en.wikisource.org/wiki/The_Story_of_an_Hour) that you will download from [this link](https://opencourses.univ-cotedazur.fr/mod/resource/view.php?id=8460).

You can save this file wherever you want (google drive, google colab temporary files, local hard disk...). Select and modify the next cells depending your way of downloading the file (and working).

In [ ]:
# this cell only if you are running this notebook on google colab and need to connect to your drive where you may have files to load.
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


### Reading the data

In [ ]:
# change according to where you decided to save the text file
pathAndFilename = "./TheStoryOfAnHour.txt"
with open(pathAndFilename, "r", encoding="utf-8") as f:
    textTSH = f.read()

print("Total number of character:", len(textTSH))

# Let's add this text to the corpus:
corpus.append(textTSH)

Total number of character: 5586


You can add later more short stories from [wikisource](https://en.wikisource.org/wiki/Category:Short_story_authors).

### Computing again the BPE vocabulary

In [ ]:
# your work

## Computing words, pairs and frequencies

### Computing words

BPE is based on frequencies of pairs of tokens **inside words**. By words, we mean a pre-tokenization of each texts in the corpus. Each text is splitted according to punctuation and spaces. Spaces are replaced by character 'Ġ' and '\n' are replaced by 'Ċ'.

🤔 <b><font color='purple'>Question:</font></b> Compute the list of words by splitting the text into words and adding a 'Ġ' before each word that follow a space character.  Don't forget to lowercase the text.

Example: "Welcome to the 2025 Deep Learning School! Deep Learning is fun. Hope you will learn a lot at the school!" will be splitted as:
<pre>['welcome', 'Ġto', 'Ġthe', 'Ġ2025', 'Ġdeep', 'Ġlearning', 'Ġschool', '!', 'Ġdeep', 'Ġlearning', 'Ġis', 'Ġfun', '.', 'Ġhope', 'Ġyou', 'Ġwill', 'Ġlearn', 'Ġa', 'Ġlot', 'Ġat', 'Ġthe', 'Ġschool', '!']</pre>

In [ ]:
# your work

In [ ]:
# hint: start using only the first text in the corpus

In [ ]:
# hint 2: a way of replacing multiple space by a single one
txt = " test 1  2   3     4                  8"
re.sub(r'(\s\s*)',' ',txt)

🤔 <b><font color='purple'>Question:</font></b> Compute the list of unique words and their occurrences in the whole corpus. `words` is the list of unique words (with "Ġ" at the beginning if the words followed a space character). `freqs` is the list counting the occurence of each word.

In [ ]:
# your work

### Pairs inside words

🤔 <b><font color='purple'>Question:</font></b> Compute all the pairs of tokens inside each of the words and their occurences. For this purpose, we suggest to use the [`defaultdict`](https://docs.python.org/3/library/collections.html#collections.defaultdict) class from `collection` which is a subclass of `dict` with the possibility of default values for missing entries. For example, if the corpus was "That thing", the pairs and occurences `pairsANDfreqs` would be the following dictionary:
{('t', 'h'): 2
('h', 'a'): 1
('a', 't'): 1
('Ġ', 't'): 1
('h', 'i'): 1
('i', 'n'): 1
('n', 'g'): 1}.
Use the real corpus, which is composed of the text from the beginning of the lab and the The_Story_of_an_Hour text.

In [ ]:
from collections import defaultdict

pairsANDfreqs = defaultdict(int)
# your work

In [ ]:
# CORRECTION
from collections import defaultdict
pairsANDfreqs = defaultdict(int)
for word, freq in zip(words,freqs):
    if len(word)>1:
        for i in range(len(word)-1):
            aPair = (word[i],word[i+1])
            pairsANDfreqs[aPair] += freq



In [ ]:
for i, key in enumerate(pairsANDfreqs.keys()):
    print(f"{key}: {pairsANDfreqs[key]}")
    if i >= 20: # not displaying everything
        break

('f', 'r'): 11
('r', 'e'): 63
('e', 'e'): 22
('g', 'o'): 2
('k', 'i'): 6
('i', 'l'): 17
('l', 'l'): 26
('l', 'e'): 30
('e', 'd'): 46
('l', 'o'): 24
('o', 'u'): 36
('u', 'i'): 6
('i', 's'): 25
('s', 'e'): 38
('w', 'e'): 15
('e', 'l'): 30
('l', 'c'): 2
('c', 'o'): 18
('o', 'm'): 22
('m', 'e'): 26
('Ġ', '2'): 1


🤔 <b><font color='purple'>Question:</font></b> Compute the most frequent pair and display it.

In [ ]:
# your work

In [ ]:
# CORRECTION
fmax = 0
pmax = None
for p,f in pairsANDfreqs.items():
    if pmax == None or f>fmax:
        pmax = p
        fmax = f
print(f"Most frequent pair: {pmax}: {fmax}")

Most frequent pair: ('h', 'e'): 176


### Merging most frequent pair of tokens

🤔 <b><font color='purple'>Question:</font></b> Next step in the BPE algorithm is to merge the elements of the most frequent pair into a single elements and to add this element to the vocabulary. Compute the merges to perform, and update the vocabulary. For example, if the corpus was "That thing", the most frequent pair would be ("t", "h"), therefore the merges would be {('t', 'h'): 'th'}, and the vocabulary (that was first initialized by individual character and the "[EOT]" token) would then contain the additional "th" element.

In [ ]:
# your work

🤔 <b><font color='purple'>Question:</font></b> At this point in the BPE algorithm, words won't be anymore splitted by characters but using the vocabulary that will grow according to most frequent pairs. We then need to build a list of splitted words `splittedWords` and update it with this new merge. For example, if the corpus was "That thing", the `splittedWords` would be initialized by {'that': ['t', 'h', 'a', 't'], 'Ġthing': ['Ġ', 't', 'h', 'i', 'n', 'g']}, but since the most frequent pair is "th", it would then be modified to become {'that': ['th', 'a', 't'], 'Ġthing': ['Ġ', 'th', 'i', 'n', 'g']}. Here, we will use the real corpus.

In [ ]:
# initial splitting of words that should then be modified
splittedWords = {word: [c for c in word] for word in words}

# for each word containing the elements that have to be merged, modify the splitted version
# In our case (real corpus), 'h' and 'e' will be merged as 'e' so that 'the' is not decomposed to 't','h' and 'e' anymore
#   but to 't' and 'he'.

# your work here

### BPE loop: refactoring code into functions

🤔 <b><font color='purple'>Question:</font></b> In the BPE encoding algorithm, we will loop on finding the most frequent pair, merging the elements of the pair and updating the splitting of each words. For this convenience, we will rewrite the previous questions using functions. As an example, you could have a function for: computing the list of words in a text, computing the list of unique words with the list of their occurence in a corpus of text, initializing the vocabulary with individual letters and "<|endoftext|>" token, computing the pairs and their occurences in words, merging a pair, computing the most frequent pair.

In [ ]:
# your work

🤔 <b><font color='purple'>Question:</font></b> Write the loop that computes the most frequent pairs and merges until a fix size of the vocabulary is reached (then, we stop the merges).

In [ ]:
# your work

## Refactoring the code into a BPETokenizer class

🤔 <b><font color='purple'>Question:</font></b> You are know ready to write the second tokenizer class of this lab: the `BPETokenizer` class. With the help of the previously defined functions, fill the body of the following class:

In [ ]:
# your work
class BPETokenizer:
    VOCAB_MAX_LENGTH = 100 # for construction

    # initialisation from a corpus of texts, building vocabulary and merges
    def __init__(self, corpus):

    # encode a single text
    def encode(self, text):

    # decode an encoded text
    def decode(self, ids):


In [ ]:
myTokenizer = BPETokenizer(corpus)
encoded = myTokenizer.encode("Welcome Deep Learning School")
print('encoded: ', encoded)
decoded = myTokenizer.decode(encoded)
print('decoded: ', decoded)

encoded:  [35, 73, 15, 81, 17, 63, 17, 17, 28, 64, 17, 75, 26, 57, 43, 15, 20, 27, 27, 24]
decoded:  welcome deep learning school


### Byte-Level BPE

So far, we have worked using letters and groups of letters so that the output was easy to read for a human. The Byte-Level BPE consider the bytes of the utf-8 encoding of words. For example:

In [ ]:
tokens = ("Voilà").encode("utf-8")
print(tokens)
tokens = list(map(int, tokens))
print(tokens)

tokens = ("Voila").encode("utf-8")
print(tokens)
tokens = list(map(int, tokens))
print(tokens)

b'Voil\xc3\xa0'
[86, 111, 105, 108, 195, 160]
b'Voila'
[86, 111, 105, 108, 97]


The initial vocabulary is then composed of the 256 possible values for 1 byte. Characters such as 'à' that are decomposed using 2 bytes in UTF-8 will be first encoded using 2 tokens encodings. If they appear frequently, they might be merged as a pair that could also be merged with another one. However, 160 and 195 may be merged independently with other values.

🤔 <b><font color='purple'>Question:</font></b> Write the Byte-level BPE tokenizer class.

In [ ]:
# your work

## Implementations of BPE

Many library and also many tokenizer algorithms exist. We present here 2 main implementation of BPE:

### Implementation from the `tiktoken` library

First play with the [tiktoken interactive application](https://tiktokenizer.vercel.app/?model=gpt2)!

![tiktoken screenshot](http://www.i3s.unice.fr/~lingrand/screenShotTiktoken.png)

Then code using the python library:

In [ ]:
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("Welcome to the 2025 Deep Learning School!")

[14618, 284, 262, 32190, 10766, 18252, 3961, 0]

### Implementation from Hugging Face

In [ ]:
# If necessary:
!pip install transformers[sentencepiece]


In [ ]:
# Loading the GPT2 tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.encode("Welcome to the 2025 Deep Learning School!")


In [ ]:

decoded_text_improved = ""
for i, token_id in enumerate(encoded_text):
    token = id2token[token_id]
    # Simple rule: Add a space before most tokens, except for punctuation that usually follows a word.
    if token not in ['.', ',', '!', '?', ':', ';', "'s", "'t", "'ll", "'d", "'ve", "'re", "'m"] and i > 0:
        decoded_text_improved += " "
    decoded_text_improved += token

# Capitalize the first letter and capitalize after periods.
decoded_text_improved = decoded_text_improved.capitalize()
sentences = decoded_text_improved.split('. ')
decoded_text_improved = '. '.join([sentence.capitalize() for sentence in sentences])

print(decoded_text_improved)

Welcome to the 2025 deep learning school  deep learning is fun  hope you will learn a lot at the school 
